# NLP Sentiment Analysis Experimentation notebook

##### Jupyter helpers:

In [1]:
%reload_ext autoreload
%autoreload

Define imports

In [2]:

from src.data import DataLoader
from src.models import SentimentClassifier
from src.data_processing import DataProcessor
from src.data_processing.text import SpacyTextProcessor
from src.experimentation import MlflowExperimentation
from src.evaluation import Evaluator, EvaluationMetrics


/Users/katyamust/anaconda3/envs/cont_review/lib/python3.7/site-packages/nltk/decorators.py:68: DeprecationWarning: `formatargspec` is deprecated since Python 3.5. Use `signature` and the `Signature` object directly
  regargs, varargs, varkwargs, defaults, formatvalue=lambda value: ""
/Users/katyamust/anaconda3/envs/cont_review/lib/python3.7/site-packages/nltk/lm/counter.py:15: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Sequence, defaultdict


## Load data
*replace MyDataLoader with your DataLoader implementation*

In [3]:
from src.data import NLPSampleDataLoader

data_loader = NLPSampleDataLoader("imdb", 1.0)
data_loader.download_dataset()
imdb_df_train, imdb_df_test = data_loader.get_dataset()

X_train, y_train = imdb_df_train['text'], imdb_df_train['label']
X_test, y_test = imdb_df_test['text'], imdb_df_test['label']

Define experimentation object, which will be used for logging the experiments parameters, metrics and artifacts
*Replace MlflowExperimentation if you use a different experimentation system*

Create preprocessor for handling data preprocessing, feature engineering etc.

In [4]:
from src.data_processing.text import SpacyTextProcessor

preprocessor = SpacyTextProcessor()

Create model/logic:

In [5]:
my_model = SentimentClassifier(preprocessor = preprocessor)

Define evaluation

In [6]:
class MyEvaluator(Evaluator):
    def evaluate(self, **kwargs) -> EvaluationMetrics:
        pass

evaluator = MyEvaluator()

Run experiment

In [7]:
from src.evaluation import EvaluationMetrics, Evaluator

from sklearn.metrics import accuracy_score

class NLPSampleEvaluationMetrics(EvaluationMetrics):
    """
    Class to hold the actual values the evaluation created, e.g. precision, recall, MSE.
    """
    def __init__(self, train_score, validation_score):
        self.train_score = train_score
        self.validation_score = validation_score
        super().__init__()

    def get_metrics(self):
        return {"train_score": self.train_score, "validation_score": self.validation_score}
    
    def __repr__(self):
        return f"train_score: {self.train_score}, validation_score: {self.validation_score}"


class NLPSampleEvaluator(Evaluator):
    """
    Class to hold the logic for how the model is evaluated.
    """
    def __init__(self, exp_train_score, exp_validation_score):
        
        # Mock values. Real values are calculated in the evaluate method
        self.exp_train_score = exp_train_score
        self.exp_validation_score = exp_validation_score
        super().__init__()

    def evaluate(self, predicted, actual) -> NLPSampleEvaluationMetrics:
        # This is where actual evaluation takes place. In the mock case it just returns constant values
        
        val_score = accuracy_score(actual, predicted)
        return NLPSampleEvaluationMetrics(
            train_score=2, validation_score=val_score
        )


evaluator = NLPSampleEvaluator(exp_train_score=0.5, exp_validation_score=0.7)


In [8]:
mlflow_experimentation = MlflowExperimentation(tracking_uri="databricks")

In [10]:
from src import ExperimentRunner



experiment_runner = ExperimentRunner(
    model=my_model,
    X_train=X_train,
    y_train=y_train,
    X_test=X_test,
    y_test=y_test,
    data_loader=data_loader,
    log_experiment=True,
    experiment_logger=mlflow_experimentation,
    evaluator=evaluator,
    experiment_name="/NewExpr",
)

results = experiment_runner.run()
print(results)


INFO: '/NewExpr' does not exist. Creating a new experiment
Logging package in /Users/katyamust/anaconda3/envs/cont_review/lib/python3.7/site-packages/nlp_sentiment_analysis-0.1.0-py3.7.egg/src


200it [00:05, 38.15it/s]
200it [00:05, 39.39it/s]


train_score: 2, validation_score: 0.765
